In [1]:
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126
# !pip install pandas
# !pip install -U sentence-transformers
# !pip install --upgrade transformers

In [2]:
# !pip show datasets

In [3]:
# !pip uninstall torchvision
# !pip install torchvision


In [1]:
import torch
print(torch.__version__)  
print(torch.version.cuda)  
print(torch.cuda.is_available())  
import torch
print(torch.__config__.show())
print(torch.cuda.device_count())  
for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")  
print(torch.backends.cudnn.enabled)  


2.6.0+cu126
12.6
True
PyTorch built with:
  - GCC 11.2
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2023.1-Product Build 20230303 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.5.3 (Git Hash 66f0cb9eb66affd2da3bf5f8d897376f04aae6af)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 12.6
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-gencode;arch=compute_86,code=sm_86;-gencode;arch=compute_90,code=sm_90
  - CuDNN 90.5.1
  - Magma 2.6.1
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, COMMIT_SHA=2236df1770800ffea5697b11b0bb0d910b2e59e1, CUDA_VERSION=12.6, CUDNN_VERSION=9.5.1, CXX_COMPILER=/opt/rh/gcc-toolset-11/root/usr/bin/c++, CXX_FLAGS= -D_GLIBCXX_USE_CXX11_ABI=1 -fvi

In [1]:
import pickle

with open('structured_abstract_sections.pkl', 'rb') as f:
    normalized_sections = pickle.load(f)

normalized_sections 


defaultdict(list,
            {'methods': ['a case report',
              'acquisition of evidence',
              'action',
              'actions',
              'activities',
              'admission findings',
              'ams subject classification',
              'analyses',
              'analysis',
              'analytical approach',
              'analytical techniques',
              'anatomy',
              'anesthesia',
              'animal',
              'animal or sample population',
              'animal population',
              'animal studied',
              'animal studies',
              'animal(s)',
              'animals',
              'animals and interventions',
              'animals and methods',
              'animals or sample population',
              'animals studied',
              'animals, materials and methods',
              'animals, methods',
              'approach',
              'approach and methods',
              'area covered',
      

In [ ]:
# def jaccard_similarity(set1, set2):
#     intersection = len(set1 & set2)
#     union = len(set1 | set2)
#     return intersection / union if union != 0 else 0

# jaccard_similarity({'Results', 'Results'}, {'Other', 'Conclusion'})


0.0

In [2]:
import random
from itertools import combinations
import pandas as pd


# Generate pairs with labels
pairs = []
for category, texts in normalized_sections.items():
    # Pairs within the same category
    category_pairs = list(combinations(texts, 2))
    sampled_pairs = random.sample(category_pairs, min(len(category_pairs), 20000))  # Randomly sample up to 5 pairs
    pairs.extend([(pair[0], pair[1], 1) for pair in sampled_pairs])  # Ensure label 1 for same-category pairs


# Pairs across different categories
categories = list(normalized_sections.keys())
for i in range(len(categories)):
    for j in range(i + 1, len(categories)):
        cross_category_pairs = [(text1, text2, 0) for text1 in normalized_sections[categories[i]] for text2 in normalized_sections[categories[j]]]
        pairs.extend(random.sample(cross_category_pairs, min(len(cross_category_pairs), 20000)))  # Randomly sample up to 5 pairs

random.shuffle(pairs)

len(pairs)

300000

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
from sentence_transformers import SentenceTransformer, InputExample, losses

class SentencePairDataset(Dataset):
    def __init__(self, sentence_pairs):
        self.samples = [InputExample(texts=[s1, s2], label=float(score)) for s1, s2, score in sentence_pairs]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]


# 建立 Dataset
train_dataset = SentencePairDataset(pairs)

# sentence-transformers 需要一個特殊的 DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)


In [7]:
from sentence_transformers import SentenceTransformer, losses

model = SentenceTransformer("all-MiniLM-L6-v2")
loss = losses.ContrastiveLoss(model)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
from datasets import Dataset

In [9]:
# 訓練模型
model.fit(
    train_objectives=[(train_dataloader, loss)],
    epochs=3,
    warmup_steps=100
)


Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss
500,0.028500
1000,0.014100
1500,0.008100
2000,0.004700
2500,0.003000
3000,0.001900
3500,0.001300
4000,0.001200
4500,0.000800
5000,0.000700


In [17]:
# !pip install transformers[torch]

In [10]:
# 載入模型
model.save("model/fine_tuned_sentence_bert_model_ContrastiveLoss")
fine_tuned_model = SentenceTransformer("fine_tuned_sentence_bert_model_ContrastiveLoss")

# 測試新模型
sentences = ["app-group", "Clinical trials assessing bsAbs targeting immunomodulatory checkpoints"]
embeddings = fine_tuned_model.encode(sentences)

print(embeddings.shape)  # (2, 384) -> 2 個句子，每個有 384 維度的嵌入


(2, 384)


In [11]:
embeddings

array([[ 3.93258817e-02,  4.30544540e-02, -6.25268742e-02,
        -8.26053414e-03, -2.00889576e-02, -9.97741055e-03,
         2.25593112e-02,  4.03656997e-02, -3.72369774e-02,
         2.76826434e-02,  5.96056171e-02, -6.89728931e-02,
        -1.94166098e-02,  5.45905270e-02, -4.08720709e-02,
        -4.99066822e-02, -6.63126633e-02,  6.37083352e-02,
        -9.60791335e-02,  1.74471308e-02,  6.89155906e-02,
        -2.29960810e-02, -2.00601611e-02, -4.56585772e-02,
        -5.71784191e-02,  9.74487960e-02,  4.75335792e-02,
         8.17093719e-03, -3.38094011e-02, -1.82331800e-01,
        -5.47954664e-02,  3.15426588e-02,  1.81849096e-02,
         1.87014844e-02, -6.04676828e-02,  1.69504192e-02,
         5.89979589e-02, -1.74948480e-02,  6.69426173e-02,
         3.01873349e-02, -5.24496809e-02, -7.57601783e-02,
        -1.15703391e-02, -5.04892543e-02,  6.72276616e-02,
         8.90713464e-03,  4.23532724e-02, -1.87274348e-02,
         2.55284626e-02,  1.37192814e-03, -8.11893791e-0